In [ ]:
import tensorflow as tf
from ovejero import bnn_alexnet
import numpy as np

### Stitch weights from one model into another

From diag model to full model

In [ ]:
img_dim = 128
num_params = 8

# These parameters aren't important but are needed to intialize the model
kr = 1e-6
dr = 1e-6
random_seed = 1234

# Initialize the two types of models
num_outputs = num_params*2
diag_model = bnn_alexnet.concrete_alexnet((img_dim, img_dim, 1), num_outputs,
    kernel_regularizer=kr,dropout_regularizer=dr,random_seed=random_seed)

num_outputs = num_params + int(num_params*(num_params+1)/2)
full_model = bnn_alexnet.concrete_alexnet((img_dim, img_dim, 1), num_outputs,
    kernel_regularizer=kr,dropout_regularizer=dr,random_seed=random_seed)

In [ ]:
# Load the weights into the diagonal model
load_weights_path = '/home/swagnercarena/ovejero/models/t1.h5'
save_weights_path = '/home/swagnercarena/ovejero/models/t2.h5'
diag_model.load_weights(load_weights_path)

In [ ]:
# Now, for all but the last layer, use the diag model weights to set the full covariance model weights
for lay_i in range(len(diag_model.layers)-1):
    get_layer = diag_model.layers[lay_i]
    set_layer = full_model.layers[lay_i]
    set_layer.set_weights(get_layer.get_weights())

In [ ]:
# Quickly check that it all worked the way you wanted it to
for lay_i in range(len(diag_model.layers)-1):
    diag_layer = diag_model.layers[lay_i]
    full_layer = full_model.layers[lay_i]
    for weight_i in range(len(diag_layer.weights)):
        assert np.sum(np.abs(full_layer.weights[weight_i].numpy()-diag_layer.weights[weight_i].numpy())) == 0

In [ ]:
# Now finally save the weights to the new path
full_model.save(save_weights_path)

From full model to diag model

In [ ]:
img_dim = 128
num_params = 8

# These parameters aren't important but are needed to intialize the model
kr = 1e-6
dr = 1e-6
random_seed = 1234

# Initialize the two types of models
num_outputs = num_params + int(num_params*(num_params+1)/2)
full_model = bnn_alexnet.concrete_alexnet((img_dim, img_dim, 1), num_outputs,
    kernel_regularizer=kr,dropout_regularizer=dr,random_seed=random_seed)

num_outputs = 2*(num_params + int(num_params*(num_params+1)/2))+1
gmm_model = bnn_alexnet.concrete_alexnet((img_dim, img_dim, 1), num_outputs,
    kernel_regularizer=kr,dropout_regularizer=dr,random_seed=random_seed)

In [ ]:
# Load the weights into the diagonal model
load_weights_path = '/home/swagnercarena/ovejero/models/t2.h5'
save_weights_path = '/home/swagnercarena/ovejero/models/t3.h5'
full_model.load_weights(load_weights_path)

In [ ]:
# Now, for all but the last layer, use the diag model weights to set the full covariance model weights
for lay_i in range(len(full_model.layers)-1):
    get_layer = full_model.layers[lay_i]
    set_layer = gmm_model.layers[lay_i]
    set_layer.set_weights(get_layer.get_weights())

In [ ]:
# Quickly check that it all worked the way you wanted it to
for lay_i in range(len(full_model.layers)-1):
    full_layer = full_model.layers[lay_i]
    gmm_layer = gmm_model.layers[lay_i]
    for weight_i in range(len(full_layer.weights)):
        assert np.sum(np.abs(gmm_layer.weights[weight_i].numpy()-full_layer.weights[weight_i].numpy())) == 0

In [ ]:
# Now finally save the weights to the new path
gmm_model.save(save_weights_path)